# Advent of code 2024

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import backend

backend.run_results = list()

# Day 1 : <a href="https://adventofcode.com/2024/day/1">Historian Hysteria</a>

## Part 1

For our first problem we will need to sort two lists, and take the sum of each pair of numbers in the sorted lists. Again, I will use my file parser to convert the input to some tuples, and convert them into a NumPy array.

In [2]:
test_input = np.array(backend.parseInput("""3   4
4   3
2   5
1   3
3   9
3   3""", backend.parseInts))

def day_1_part_1(input : np.array):
    sorted_input = np.sort(input, axis = 0)
    return(np.abs(sorted_input[:,1] - sorted_input[:, 0]).sum())

backend.test(day_1_part_1, test_input, 11)


__________ Input to be parsed __________
3   4
4   3
2   5
1   3
3   9
... and maybe more
____________________
__________ Parsed input __________
(3, 4)
(4, 3)
(2, 5)
(1, 3)
(3, 9)
... and maybe more
____________________
Answer: 11            0.000172 seconds
Test succeeded.


In [3]:
real_input = np.array(backend.parseInput(open("inputs/day1.txt").read(), backend.parseInts))

backend.run(day_1_part_1, real_input, 1, 1)

__________ Input to be parsed __________
76309   75213
79731   28444
29583   71339
60992   99148
34680   74530
... and maybe more
____________________
__________ Parsed input __________
(76309, 75213)
(79731, 28444)
(29583, 71339)
(60992, 99148)
(34680, 74530)
... and maybe more
____________________
Answer for day 1 part 1: 1110981            0.000378 seconds


1110981

That was easy! Now on to part 2.

## Part 2

It turns out that our first list contains ID's and we need to count each ID in the second list. We multiply the ID by the number of occurences in the second list, to obtain the <b>Similarity score</b>.

In [4]:
def day_1_part_2(input : np.array):
    first_list, second_list = input[:, 0], input[:, 1]
    frequency_list = np.array([np.sum(second_list == key) for key in first_list])
    return np.sum(first_list * frequency_list)

backend.test(day_1_part_2, test_input, 31)

Answer: 31            0.000709 seconds
Test succeeded.


In [5]:
backend.run(day_1_part_2, real_input, 1, 2)

Answer for day 1 part 2: 24869388            0.014192 seconds


24869388

Day 1 was quite doable, as ever with AOC. Let's see what day 2 brings us!

# Day 2 : <a href="https://adventofcode.com/2024/day/2">Red-Nosed Reports</a>

## Part 1


In [52]:
test_input = backend.parseInput("""7 6 4 2 1
1 2 7 8 9
9 7 6 2 1
1 3 2 4 5
8 6 4 4 1
1 3 6 7 9""", parseMethod = backend.parseInts)

def is_safe(t : tuple):
    diff_arr = np.diff(t)
    monotonic = min(diff_arr) * max(diff_arr) > 0
    safe_abs_change = max(np.abs(diff_arr)) <= 3
    return(monotonic * safe_abs_change)

def day_2_part_1(input):
    map_obj = map(is_safe, [data for data in input])
    return(sum(map_obj))


# backend.test(day_2_part_1, test_input, 2)
day_2_part_1(test_input)

__________ Input to be parsed __________
7 6 4 2 1
1 2 7 8 9
9 7 6 2 1
1 3 2 4 5
8 6 4 4 1
... and maybe more
____________________
__________ Parsed input __________
(7, 6, 4, 2, 1)
(1, 2, 7, 8, 9)
(9, 7, 6, 2, 1)
(1, 3, 2, 4, 5)
(8, 6, 4, 4, 1)
... and maybe more
____________________


2

In [53]:
real_input = backend.parseInput(open("inputs/day2.txt").read(), parseMethod = backend.parseInts)
backend.run(day_2_part_1, real_input, 2, 1)

__________ Input to be parsed __________
20 21 24 25 27 29 27
60 61 62 64 64
15 18 21 22 25 26 30
5 8 11 14 16 19 20 26
20 22 20 23 24
... and maybe more
____________________
__________ Parsed input __________
(20, 21, 24, 25, 27, 29, 27)
(60, 61, 62, 64, 64)
(15, 18, 21, 22, 25, 26, 30)
(5, 8, 11, 14, 16, 19, 20, 26)
(20, 22, 20, 23, 24)
... and maybe more
____________________
Answer for day 2 part 1: 526            0.018712 seconds


526

In [ ]:
def sign_swaps(a : np.array):
    """Return indices of where the sign of a np array swaps"""
    return(np.array([i for i in np.arange(1, len(a)) if a[i] * a[i-1] < 0]))

def is_safe(t : tuple):
    diff_t = np.diff(t)
    unsafe_abs_changes = np.where(np.abs(diff_t) > 3)
    if np.any(unsafe_abs_changes) : return False
    non_monotonic_locs = sign_swaps(diff_t)
    if len(non_monotonic_locs) <= 1 : return True
    if len(non_monotonic_locs) == 2  and np.diff(non_monotonic_locs).max() <= 1: return True


is_safe(np.array(test_input[3]))


# Statistics AOC 2024

In [6]:
backend.neat_results()

,Day,Part,Result,Time
0,1,1,1110981,0.000378
1,1,2,24869388,0.014192
